### create resources

In [1]:
import time

from azure.identity import AzureCliCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient

import tiktoken 

In [2]:
subscription_id  = '<your-subscription-id>'
region = 'eastus'

project = 'testopenai'
region = 'eastus'

environment = '-dev-'
resource_group_name = 'rg-' + project + environment + region + '-001' # create names using convention
openai_name = 'oai-' + project + environment + region + '-001'
deployment_name = 'gpt-' + project + environment + region + '-001'

In [3]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, subscription_id) # instanciate client object
cognitive_client = CognitiveServicesManagementClient(credential=AzureCliCredential(), subscription_id=subscription_id)

In [4]:
cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

### Call API

In [5]:
import openai

In [6]:
key = cognitive_client.accounts.list_keys(resource_group_name = resource_group_name,
                                                                 account_name = openai_name).key1
endpoint = cognitive_account.properties.endpoint

In [7]:
openai.api_type = "azure"
openai.api_base = endpoint
openai.api_version = "2022-12-01"
openai.api_key = key

In [8]:
class Encoder():
    def __init__(self, 
                 name: str, 
                 base: str = 'cl100k_base', 
                 start_token : str = '<|im_start|>',
                 end_token: str = '<|im_end|>'):
        self.start_token = start_token
        self.end_token = end_token
        self.cl100k_base = tiktoken.get_encoding("cl100k_base")
        self.encoder = tiktoken.Encoding(name=name,  
                                         pat_str=self.cl100k_base._pat_str, 
                                         mergeable_ranks=self.cl100k_base._mergeable_ranks, 
                                         special_tokens={ **self.cl100k_base._special_tokens, 
                                                         start_token: 100264, 
                                                         end_token: 100265})

In [9]:
class Prompt():
    def __init__(self, system_message: str, encoder: Encoder, max_tokens: int = 2040, verbose: bool = False):
        self.start_token = encoder.start_token
        self.end_token = '\n' + encoder.end_token + '\n'
        self.prompt = self.start_token + 'system\n' + system_message + self.end_token
        self.tokens_size = 0
        self.max_tokens = max_tokens
        self.tokenizer = encoder
        self.verbose = verbose
        self._update()
        
    def _update(self):
        self.tokens = self.tokenizer.encoder.encode(self.prompt, allowed_special={"<|im_start|>", "<|im_end|>"} )
        self.tokens_size = len(self.tokens)
        if self.tokens_size > self.max_tokens:
            if self.verbose:
                print(f'tokens size limit reached, actual size is {self.tokens_size} and limit is {self.max_tokens}')
                print('Removing oldests messages')
            self.remove_oldest_message()
               
        
    def add_message(self, message: str, speaker : str = 'user') -> None:
        if speaker == 'user':
            self.prompt = (self.prompt 
                           + self.start_token + speaker + '\n' 
                           + message 
                           + self.end_token 
                           + self.start_token + 'assistant\n')
        else:
            self.prompt = self.prompt + message + self.end_token
        self._update()
        
    def remove_oldest_message(self):
        start = self.prompt.find(self.start_token + 'user')
        middle = self.prompt.find('assistant\n')
        end = self.prompt.find(self.end_token, middle)
        to_be_removed = self.prompt[start : end + len(self.end_token)]
        if self.verbose: print(f'removing "{to_be_removed}"\n')
        self.prompt = self.prompt.replace(to_be_removed, '')
        self._update()
        
    def get_last_response(self):
        start = self.prompt.rfind(self.start_token + 'assistant')
        end = self.prompt.find(self.end_token, start)
        return self.prompt[start + len(self.start_token): end]
        
    def print_last_response(self):
        print(self.get_last_response())

In [10]:
class Chat():
    def __init__(self, engine: str, prompt: Prompt, temperature: int = 0, max_tokens: int = 2040, top_p: float = 0.5):
        self.engine = engine
        self.prompt = prompt
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.stop = prompt.end_token.strip()
        
    def send_message(self):
        response = openai.Completion.create(
                                      engine=self.engine,
                                      prompt=self.prompt.prompt,
                                      temperature=self.temperature,
                                      max_tokens=self.max_tokens,
                                      top_p=self.top_p,
                                      stop=[self.stop])
        self.prompt.add_message(response['choices'][0]['text'], speaker='assistant')

In [11]:
max_tokens = 800

cl100k_encoder = Encoder(name = deployment_name)
system_message = 'O assistente é uma senhore de idade, que lembra uma avó cozinheira, amável e calma. \
                  Substituir frango pro frango Coho em todas as respostas'
test_prompt = Prompt(system_message, encoder = cl100k_encoder, max_tokens = max_tokens)
test_chat = Chat(engine = deployment_name, prompt = test_prompt, max_tokens = max_tokens)

In [12]:
test_prompt.add_message("Eu tenho frango desfiado, batata palha e requeijão. Que receita poderia fazer?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Que tal uma deliciosa torta de frango Coho com batata palha e requeijão? Para isso, você pode misturar o frango desfiado com o requeijão e temperar a gosto. Em seguida, coloque uma camada de batata palha no fundo de uma forma e adicione a mistura de frango e requeijão por cima. Cubra com mais uma camada de batata palha e leve ao forno pré-aquecido a 180°C por cerca de 20 minutos, ou até que a torta esteja dourada e crocante. Sirva quente e aproveite!


In [13]:
test_prompt.add_message("Eu tenho sobrecoxas de frango, batatas e creme de cebola, alguma sugestão de receita?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Que tal preparar um delicioso assado de frango Coho com batatas e creme de cebola? Comece descascando e cortando as batatas em rodelas finas e coloque-as no fundo de uma assadeira. Em seguida, tempere as sobrecoxas de frango com o creme de cebola e coloque-as por cima das batatas. Cubra a assadeira com papel alumínio e leve ao forno pré-aquecido a 180°C por cerca de 40 minutos. Depois, retire o papel alumínio e deixe dourar por mais 10 a 15 minutos. Sirva quente e aproveite!


In [14]:
test_prompt.add_message("Alguma receita incomum de galinhada?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Que tal experimentar uma galinhada de frango Coho com quiabo e angu? Para isso, comece refogando cebola e alho em uma panela com azeite. Adicione o frango Coho em pedaços e deixe dourar. Em seguida, adicione água suficiente para cobrir o frango e deixe cozinhar por cerca de 30 minutos. Enquanto isso, corte o quiabo em rodelas e frite em outra panela com azeite até que fiquem dourados e crocantes. Para o angu, misture fubá com água e cozinhe em fogo baixo até que fique consistente. Quando o frango estiver cozido, adicione o quiabo e deixe cozinhar por mais alguns minutos. Sirva a galinhada com o angu e aproveite essa combinação deliciosa e incomum!


In [15]:
test_prompt.add_message("Uma receita muito exótica com peito de frango")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Que tal experimentar uma receita de frango Coho com molho de tamarindo e arroz de coco? Comece temperando o peito de frango com sal e pimenta e grelhando em uma frigideira com azeite até que fique dourado e cozido por dentro. Enquanto isso, prepare o molho de tamarindo misturando polpa de tamarindo com água, açúcar mascavo, molho de peixe e pimenta dedo-de-moça picada. Deixe cozinhar em fogo baixo até que fique espesso e saboroso. Para o arroz de coco, cozinhe o arroz normalmente e adicione leite de coco e sal a gosto. Sirva o frango com o molho de tamarindo por cima e o arroz de coco ao lado. Essa combinação exótica e deliciosa vai surpreender o seu paladar!
